### doc_entity
ESTABELECE RELAÇÕES ENTRE DOCUMENTOS E ENTIDADES NA BASE DE DADOS NO MYSQL. TRABALHAMOS COM O ACERVO **ANTONIO AZEREDO DA SILVEIRA, MINISTÉRIO DAS RELAÇÕES EXTERIORES**.  

AS ENTIDADES A SEREM TRABALHADAS SÃO:
* PAÍSES
* PESSOAS

In [1]:
import os
import sys
import time
import datetime
import numpy as np
import pandas as pd
import re

#import pymysql
import getpass
#from sshtunnel import SSHTunnelForwarder

from IPython.display import clear_output

import csv
import xlrd
import xlwt

import getpass
from sshtunnel import SSHTunnelForwarder
import pymysql

import unicodedata
import numpy as np
from collections import OrderedDict, Counter
import nltk
from nltk.collocations import *
from nltk.util import ngrams
import string
import pickle

Especifica qual sistema operacional está sendo usado. Renato = Linux ; Marcelo = nt (Windows) e define variáveis a partir disso

In [2]:
if os.name == 'nt':
    encoding_type = 'utf-8'
    ssh_priv_key = 'C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa'
    ssh_user='marcelobribeiro'
    sql_user='marcelobribeiro'
    path_inputs = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/Processamento/text-learning-tools/inputs/"
    #path_inputs = "C:/Users/MARCELO/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/Processamento/text-learning-tools/inputs/"
    path_outputs = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/Processamento/text-learning-tools/outputs/"
else:
    encoding_type = 'ISO-8859-1'
    ssh_priv_key = '/home/rsouza/.ssh/id_rsa'
    ssh_user='rsouza'
    sql_user='rsouza'
    path_inputs = "/home/rsouza/Documentos/text-learning-tools/inputs/"
    path_outputs = "/home/rsouza/Documentos/text-learning-tools/outputs/"

In [3]:
countries = {}
country_names_file = path_inputs+"history-lab-country_table.csv"
with open(country_names_file, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=";")
    for row in reader:
        varnames = []
        en_name = row[0]
        pt_name = row[1]
        city = row[2]
        varnames = [en_name]
        if en_name != pt_name: 
            varnames.append(pt_name)
        if city != "":
            varnames.append(city)
        countries[en_name] = varnames

In [4]:
countries

{'afghanistan': ['afghanistan', 'afeganistão', 'kabul'],
 'aland islands': ['aland islands', 'ilhas aland'],
 'albania': ['albania', 'albânia', 'tirana'],
 'algeria': ['algeria', 'argélia', 'algiers'],
 'american samoa': ['american samoa', 'samoa americana', 'pago pago'],
 'andorra': ['andorra', 'andorra la vella'],
 'angola': ['angola', 'luanda'],
 'anguilla': ['anguilla'],
 'antarctica': ['antarctica', 'antártica'],
 'antigua and barbuda': ['antigua and barbuda',
  'antígua e barbuda',
  "st. john's"],
 'argentina': ['argentina', 'buenos aires'],
 'armenia': ['armenia', 'armênia'],
 'aruba': ['aruba'],
 'australia': ['australia', 'austrália', 'canberra'],
 'austria': ['austria', 'áustria', 'vienna'],
 'azerbaijan': ['azerbaijan', 'azerbaijão', 'baku'],
 'bahrain': ['bahrain', 'bahrein', 'manama'],
 'bangladesh': ['bangladesh', 'dhaka'],
 'barbados': ['barbados', 'bridgetown'],
 'belarus': ['belarus', 'bielorrússia'],
 'belgium': ['belgium', 'bélgica', 'brussels'],
 'belize': ['belize

In [5]:
count = 0

pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.147', 22),
                        ssh_private_key=ssh_priv_key,
                        ssh_private_key_password=pass_sshkey,
                        ssh_username=ssh_user,
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user=sql_user,
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    OBS: o ID não é VARCHAR!
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

    cur.execute("DROP TABLE IF EXISTS countries")
    cur.execute('''CREATE TABLE IF NOT EXISTS countries
               (id VARCHAR(128) PRIMARY KEY, name VARCHAR(128)
               DEFAULT NULL)
               ENGINE=MyISAM DEFAULT CHARSET='utf8';''')
    
    cur.execute("DROP TABLE IF EXISTS country_doc")
    cur.execute('''CREATE TABLE IF NOT EXISTS country_doc
               (country_id VARCHAR(128), doc_id VARCHAR(31), country_count INT(11)
               DEFAULT NULL)
               ENGINE=MyISAM DEFAULT CHARSET='utf8';''')
    
    cur.execute("SELECT * FROM CPDOC_AS.docs WHERE main_language = 'pt' AND (readability > 0.4 OR readability = -1) ")  
    
    text_data = cur.fetchall()
    
    for country,values in countries.items():
        country_id = count + 500000 # id do History-Lab termina em 116.000. Precisamos ter uma sequência de ID distante, tal como foi acordado entre Marcelo e Rohan
        
        ### mede etapa da iteração. Número de etapas = 278 ###
        clear_output()
        print("Mining country number ",count)
        
        for row_text in text_data:
            text = row_text[4]
            doc_id = row_text[0]
                        
            country_count = 0
            for name in values:
                country_subfreq = text.count(name)
                country_count += country_subfreq
            
            if country_count > 0: 
                query = "INSERT INTO country_doc VALUES (%s, %s, %s)"
                cur.execute(query, (country_id, doc_id, country_count))
        
        query = "INSERT INTO countries VALUES (%s, %s)"
        cur.execute(query, (country_id, country))
        
        count += 1
        #if count >= 100: break #amostra de apenas x linhas ~ 10%
    


Mining country number  278
